# Data prepration

In [6]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats

############
# Functions
############

# Our 3d parameter space is the surface of a sphere centered at (1, 1, 1), with radius 1/sqrt(2)
# x, y coordinates are the x and y variances of our 2d Gaussian
# z coordinate determines the x-coordinate of the mean, mu = (z, 0)

def mean_gen(theta, phi):
    mu = 0.5 - (np.sqrt(2)/2) * np.cos(phi + np.pi / 4)
    return mu


def varx_gen(theta, phi):
    vx = 1.5 - (np.sqrt(2)/2) * np.cos(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vx


def vary_gen(theta, phi):
    vy = 1.5 - (np.sqrt(2)/2) * np.sin(theta + np.pi / 4) * np.sin(phi + np.pi / 2)
    return vy


# Generate n data
def spherical_data(n, thetas, phis, rand=1234):
    
    mx1, my1 = np.zeros(n), np.zeros(n)
    vx1, vy1 = np.ones(n), np.ones(n)
    
    mx2, my2 = mean_gen(thetas, phis), np.zeros(n)
    vx2, vy2 = varx_gen(thetas, phis), vary_gen(thetas, phis)
    
    x1, y1 = np.transpose(np.array([np.random.normal(mx1, vx1, size=n), 
                                    np.random.normal(my1, vy1, size=n), thetas, phis])), np.zeros(n)
    x2, y2 = np.transpose(np.array([np.random.normal(mx2, vx2, size=n), 
                                    np.random.normal(my2, vy2, size=n), thetas, phis])), np.ones(n)
    
    x, y = np.append(x1, x2, axis=0), np.append(y1, y2, axis=0)

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = rand)
    
    return x_train, x_test, y_train, y_test


def discrete_angles(n, m, s, rand = 1234):
    angles = np.random.uniform(0, s * np.pi, m)
    xk = np.arange(m)
    pk = (1 / m) * np.ones(int(m))
    discrete_distr = stats.rv_discrete(name='discrete_distr', values=(xk, pk), seed=rand)
    thetas = angles[discrete_distr.rvs(size=n)]
    return thetas

In [7]:
n = 100000


# m_th = 40
# m_ph = 40
# thetas = discrete_angles(n, m_th, 2)
# phis = discrete_angles(n, m_ph, 1)
# name = ('discrete_model_mth%.0f_mph%.0f' %(m_th, m_ph))

theta = 2
phi = 2
thetas = theta * np.ones(n)
phis = phi * np.ones(n)
name = ('discrete_model_th%.0f_ph%.0f' %(theta, phi))

x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis)

print(x_train)

[[ 0.6966621   0.64602116  2.          2.        ]
 [ 1.21093747  0.81166126  2.          2.        ]
 [ 2.05257671  0.06592049  2.          2.        ]
 ...
 [ 0.34160153  0.66658316  2.          2.        ]
 [ 0.66673211 -3.24127035  2.          2.        ]
 [ 2.29239147 -0.44680839  2.          2.        ]]


# Training Interpolated Model

In [8]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
]);

loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

batch_size = 1000

checkpoint_path = "tutorial/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    save_freq=10*batch_size
)

log_dir = "tutorial/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

EPOCHS=20
model.fit(
    x_train, y_train, epochs=EPOCHS,
    batch_size=batch_size,
    callbacks=[cp_callback, tensorboard_callback]
);

model.save('3dmodels/' + name)

2022-08-08 12:04:58.419229: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-08 12:04:58.419265: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-08 12:04:58.419507: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/20
 20/160 [==>...........................] - ETA: 1s - loss: 0.5996 - accuracy: 0.6780

2022-08-08 12:04:58.979348: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-08 12:04:58.979385: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-08 12:04:59.005685: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-08-08 12:04:59.006705: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-08-08 12:04:59.018972: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220808-120458/train/plugins/profile/2022_08_08_12_04_59

2022-08-08 12:04:59.024017: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to tutorial/logs/20220808-120458/train/plugins/profile/2022_08_08_12_04_59/jupyter-matan.trace.json.gz
2022-08-08 12:04:59.026139: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220808-120

160/160 [==============================] - 1s 6ms/step - loss: 0.5438 - accuracy: 0.7224
Epoch 2/20
160/160 [==============================] - 1s 6ms/step - loss: 0.5330 - accuracy: 0.7305
Epoch 3/20
160/160 [==============================] - 1s 6ms/step - loss: 0.5320 - accuracy: 0.7314
Epoch 4/20
160/160 [==============================] - 1s 5ms/step - loss: 0.5315 - accuracy: 0.7308
Epoch 5/20
160/160 [==============================] - 1s 5ms/step - loss: 0.5312 - accuracy: 0.7310
Epoch 6/20
160/160 [==============================] - 1s 5ms/step - loss: 0.5314 - accuracy: 0.7305
Epoch 7/20
160/160 [==============================] - 1s 4ms/step - loss: 0.5311 - accuracy: 0.7310
Epoch 8/20
160/160 [==============================] - 1s 4ms/step - loss: 0.5313 - accuracy: 0.7317
Epoch 9/20
160/160 [==============================] - 1s 5ms/step - loss: 0.5308 - accuracy: 0.7310
Epoch 10/20
160/160 [==============================] - 1s 5ms/step - loss: 0.5308 - accuracy: 0.7310
Epoch 11/2

In [16]:
model = tf.keras.models.load_model('3dmodels/' + name)